In [ ]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.sparse import diags, csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from numpy.lib.function_base import blackman
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [ ]:
def ftcs_menno(N, M, T, S_0, S_max, K, r, sigma):
    n_space = M # grid size
    n_time = N  # time steps
    r = r
    sigma = sigma
    strike_price = K
    stock_price = S_0
    S_max = 4 * strike_price
    S_min = strike_price / 4
    t_max = 1

    x_max = np.log(S_max)
    x_min = np.log(S_min)
    x0 = np.log(stock_price)

    x, dx = np.linspace(x_min, x_max, n_space, retstep=True)
    t, dt = np.linspace(0, t_max, n_time, retstep=True)

    s = np.exp(x)
    payoff = np.maximum(s - strike_price, 0)
    v = np.zeros((n_space, n_time))
    v[:, -1] = payoff
    v[0, :] = 0
    v[-1, :] = np.exp(x_max) - strike_price * np.exp(-r * t[::-1])

    sigma2, dx2 = sigma**2, dx**2

    alpha = 0.5 * dt * ((0.5 * sigma2 - r) / dx + sigma2 / dx2)
    beta = sigma2 * dt / dx2 + r * dt
    gamma = 0.5 * dt * ((r - 0.5 * sigma2) / dx + sigma2 / dx2)

    a = -alpha
    b = 1 + beta
    c = -gamma

    A = diags([a, b, c], [-1, 0, 1], shape=(n_space-2, n_space-2)).tocsc()
    offset = np.zeros(n_space-2)

    for i in range(n_time-2, -1, -1):

        offset[0] = a * v[0, i]
        offset[-1] = c * v[-1, i]
        v[1:-1, i] = spsolve(A, (v[1:-1, i+1] - offset))

    v_ftcs = v[:, 0]
    option_value = np.interp(x0, x, v_ftcs)
    # print(f'call option value: {np.interp(x0, x, v_ftcs)}')

    return v_ftcs, option_value


ftcs_menno(500, 100, 1, 100, 200, 99, 0.06, 0.2)

In [3]:
def fd_cd(N, M, T, S_0, S_max, K, r, sigma):
    '''
    N -> Number of time steps
    M -> Number of grid spaces
    S_max -> Maximum stock price 
    '''
    dt = T / N  # Time step
    S_max = 2 * S_0
    S_max = 4 * K
    dS = S_max / M # Space step

    #  S0   [i = 1]
    #  S    [i = 2]
    #  Smax [i = M]

    all_i = np.arange(1, M)
    all_j = np.arange(N)
    all_S = np.linspace(0, S_max, M+1) # M+1 equally spaced values of S, from 0

    # Populate the grid with the the values
    grid = np.zeros(shape=(M+1, N+1))
    grid[:, -1] = np.maximum(all_S - K, 0)

    # Greek Arrays
    alpha = 0.25 * dt * (sigma**2 * all_i**2 - r * all_i)
    beta = -0.5 * dt * (sigma**2 * all_i**2 + r)
    gamma = 0.25 * dt * (sigma**2 * all_i**2 + r * all_i)

    # A and B matrices
    A = np.diag(alpha[1:], -1) + np.diag(1 + beta) + np.diag(gamma[:-1], 1)
    B = np.diag(-alpha[1:], -1) + np.diag(1 - beta) + np.diag(-gamma[:-1], 1)

    # Bottom boundary conditions (Dirichlet Condition)
    B[0,   0] -= 2*alpha[0]
    B[0,   1] += alpha[0]
    B[-1, -1] -= 2*gamma[-1]
    B[-1, -2] += gamma[-1]

    # # Side boundary conditionS
    A[0,   0] += 2*alpha[0]
    A[0,   1] -= alpha[0]
    A[-1, -1] += 2*gamma[-1]
    A[-1, -2] -= gamma[-1]

    # Terminal Condition (call option)
    grid[:, -1] = np.maximum(all_S - K, 0)

    # PLU Decomposition
    P, L, U = linalg.lu(B)
    for j in reversed(all_j):
        Ux = linalg.solve(L, np.dot(A, grid[1:-1, j+1]))
        grid[1:-1, j] = linalg.solve(U, Ux)
        grid[0, j] = 2 * grid[1, j] - grid[2, j]
        grid[-1, j] = 2 * grid[-2, j] - grid[-3, j]

    # option_value = grid[:, 0][int(len(grid)/2)]
    option_value = np.interp(S_0,all_S,grid[:, 0])
    # print(f"Estimated option value: {option_value}")
    return grid, option_value


grid, value = fd_cd(500, 100, 1, 100, 200, 99, 0.06, 0.2)
# grid, value = fd_cd(1000, 500, 1, 50, 100, 50, 0.06, 0.4)

In [4]:
def ftcs(N, M, T, S_0, S_max, K, r, sigma, optimal_delta=True):
    '''
    N -> Number of time steps
    M -> Number of grid spaces
    S_max -> Maximum stock price 
    '''
    S_max = 2 * S_0
    if optimal_delta:
        dt = 0.0005
        N = int(T/dt)
    else:
        # print(N  )
        dt = T / N  # Time step
    dS = S_max / M  # Space step
    # print(dt)
    #  S0   [i = 1]
    #  S    [i = 2]
    #  Smax [i = M]

    all_i = np.arange(1, M)
    all_j = np.arange(N)
    # M+1 equally spaced values of S, from 0
    all_S = np.linspace(0, S_max, M+1)

    # Populate the grid with the the values
    grid = np.zeros(shape=(M+1, N+1))
    grid[:, -1] = np.maximum(all_S - K, 0)

    # Greek Arrays
    alpha = (0.25 * dt * (sigma**2 * all_i**2 - r * all_i))*2
    beta = (-0.5 * dt * (sigma**2 * all_i**2 + r))*2
    gamma = (0.25 * dt * (sigma**2 * all_i**2 + r * all_i))*2

    # A matrix
    A = np.diag(alpha[1:], -1) + np.diag(1 + beta) + np.diag(gamma[:-1], 1)

    # Side boundary conditionS
    A[0,   0] += 2*alpha[0]
    A[0,   1] -= alpha[0]
    A[-1, -1] += 2*gamma[-1]
    A[-1, -2] -= gamma[-1]

    # Terminal Condition (call option)
    grid[:, -1] = np.maximum(all_S - K, 0)

    # Iterate over the grid
    for j in reversed(all_j):
            old_grid = grid.copy()
            grid[1:-1, j] = np.dot(A, grid[1:-1, j+1])
            grid[0, j] = 2 * grid[1, j] - grid[2, j]
            grid[-1, j] = 2 * grid[-2, j] - grid[-3, j]
            if np.isnan(grid[:, 0][int(len(grid)/2)]):
                    print("Abort")
                    # option_value = old_grid[:, 0][int(len(grid)/2)]
                    option_value = np.interp(S_0, all_S, grid[:, 0])
                    print(f"Estimated option value: {option_value}")
                    return old_grid, option_value

    # option_value = grid[:, 0][int(len(grid)/2)]
    option_value = np.interp(S_0, all_S, grid[:, 0])
    # print(f"Estimated option value: {option_value}")
    return grid, option_value

In [5]:
def fd_cd(N, M, T, S_0, S_max, K, r, sigma):
    '''
    N -> Number of time steps
    M -> Number of grid spaces
    S_max -> Maximum stock price 
    '''
    dt = T / N  # Time step
    S_max = 2 * S_0
    S_max = 4 * K
    dS = S_max / M # Space step

    #  S0   [i = 1]
    #  S    [i = 2]
    #  Smax [i = M]

    all_i = np.arange(1, M)
    all_j = np.arange(N)
    all_S = np.linspace(0, S_max, M+1) # M+1 equally spaced values of S, from 0

    # Populate the grid with the the values
    grid = np.zeros(shape=(M+1, N+1))
    grid[:, -1] = np.maximum(all_S - K, 0)

    # Greek Arrays
    alpha = 0.25 * dt * (sigma**2 * all_i**2 - r * all_i)
    beta = -0.5 * dt * (sigma**2 * all_i**2 + r)
    gamma = 0.25 * dt * (sigma**2 * all_i**2 + r * all_i)

    # A and B matrices
    A = np.diag(alpha[1:], -1) + np.diag(1 + beta) + np.diag(gamma[:-1], 1)
    B = np.diag(-alpha[1:], -1) + np.diag(1 - beta) + np.diag(-gamma[:-1], 1)

    # Bottom boundary conditions (Dirichlet Condition)
    B[0,   0] -= 2*alpha[0]
    B[0,   1] += alpha[0]
    B[-1, -1] -= 2*gamma[-1]
    B[-1, -2] += gamma[-1]

    # # Side boundary conditionS
    A[0,   0] += 2*alpha[0]
    A[0,   1] -= alpha[0]
    A[-1, -1] += 2*gamma[-1]
    A[-1, -2] -= gamma[-1]

    # Terminal Condition (call option)
    grid[:, -1] = np.maximum(all_S - K, 0)

    # PLU Decomposition
    P, L, U = linalg.lu(B)
    for j in reversed(all_j):
        Ux = linalg.solve(L, np.dot(A, grid[1:-1, j+1]))
        grid[1:-1, j] = linalg.solve(U, Ux)
        grid[0, j] = 2 * grid[1, j] - grid[2, j]
        grid[-1, j] = 2 * grid[-2, j] - grid[-3, j]

    # option_value = grid[:, 0][int(len(grid)/2)]
    option_value = np.interp(S_0,all_S,grid[:, 0])
    # print(f"Estimated option value: {option_value}")
    return grid, option_value


grid, value = fd_cd(500, 100, 1, 100, 200, 99, 0.06, 0.2)
# grid, value = fd_cd(1000, 500, 1, 50, 100, 50, 0.06, 0.4)


In [6]:
# For saving
main_name = "menno_edition"

# Parameters
r = 0.04
sigma = 0.3
S_max = 200
K = 110
T = 1
N = 50
M = 1000
midpoint = int(M/2)
S_0 = 100


# S_0 = 100 # In the money
# S_0 = 110 # At the money
# S_0 = 120  # Out of the money


# ----- 2D price plot ------
grid_cn, value = fd_cd(N, M, T, S_0, S_max, K, r, sigma)

all_S = np.linspace(0, S_max, M+1)
price_array_cn = []
for price in tqdm(all_S):
    grid_cn, value = fd_cd(N, M, T, price, S_max, K, r, sigma)
    price_array_cn.append(value)

grid_ftcs, value = ftcs_menno(N, M, T, price, S_max, K, r, sigma)

price_array_ftcs = []
for price in tqdm(all_S):
    grid_ftcs, value = ftcs(N, M, T, price, S_max, K, r, sigma)
    price_array_ftcs.append(value)

price_array_an = []
for price in tqdm(all_S):
    price_array_an.append(analytical_bs(T=T, S0=price, K=K, sigma=sigma, r=r))

plt.plot(all_S, price_array_an, label="BS", lw=3)
plt.plot(all_S, price_array_ftcs, label="FTCS", ls='--', lw=3)
plt.plot(all_S, price_array_cn, label="CN", lw=3, ls=":")
plt.legend()
plt.ylabel("Option price")
plt.xlabel("Stock price")
plt.tight_layout()
plt.savefig(f"{main_name}_2D_priceplot.pdf")


# ----- 3D final grid plot ------


N = 200
M = 1000
grid_cn, value = fd_cd(N, M, T, S_0, S_max, K, r, sigma)
all_S = np.linspace(0, S_max, M+1)
fig = plt.figure()
ax = fig.gca(projection='3d')
# Make data.
# THE ACTUAL DATA SHOWS THE REVERSED AXIS BUT THE AXIS LABEL DOES NOT, COMMENT THIS OUT TO SEE TRUE GRAPH
# all_S = all_S[::-1]
Y = all_S
X = np.arange(N)
# X = X.transpose()
X, Y = np.meshgrid(X, Y)
Z = grid_cn[:, :-1]
# Z = grid_ftcs[:, :-1]
# Z = Z.transpose()

print(X.shape, Y.shape, Z.shape)
# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=1, antialiased=False)

# Customize the z axis.
# ax.set_zlim(-1.01, 1.01)
# ax.zaxis.set_major_locator(LinearLocator(10))
# ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
ax.set_xlabel(xlabel="Time")
ax.set_ylabel(ylabel="Stock price")
ax.set_zlabel(zlabel="Option price")
# ax.set_title("Crank Nicholson")
# ax.invert_xaxis()

# plt.gca().invert_xaxis()
plt.tight_layout()
# plt.savefig(f"{main_name}_3D_cnplot.pdf")
plt.show()


#---------- Converge in space ------
# Show 3 different plots for 3 moneyness, repeat with more iterations and cut enough to show FTCSdevolving

#Parameters
r = 0.04
sigma = 0.3
S_max = 200
K = 110
T = 1
N = 50 # 100
results = {}
grid_sizes = np.linspace(20, 1500, 150)  # try 5,500,20
moneyness = [100, 110, 120]
for money in moneyness:
    abort = True
    all_prices_cn = []
    all_prices_ftcs = []
    all_errors_cn = []
    all_errors_ftcs = []
    for grid_size in tqdm(grid_sizes):
        BSA = analytical_bs(T=T, S0=money, K=K, sigma=sigma,
                            r=r, return_delta=False)
        grid_cn, value_1 = fd_cd(N, int(grid_size), T,
                                 money, S_max, K, r, sigma)
        # grid_ftcs, value_2 = ftcs(N, int(grid_size), T, money, S_max, K, r, sigma)
        if abort:
            grid_ftcs, value_2 = ftcs_menno(
                N, int(grid_size), T, money, S_max, K, r, sigma)
            if np.isnan(value_2):
                abort = False
                print("Nan")
        else:
            print("Nan")
            value_2 = BSA
        # Append prices
        all_prices_cn.append(value_1)
        all_prices_ftcs.append(value_2)
        all_errors_cn.append(abs(value_1-BSA)/BSA)
        all_errors_ftcs.append(abs(value_2-BSA)/BSA)
    results[money] = [all_prices_cn, all_prices_ftcs,
                      all_errors_cn, all_errors_ftcs]

# moneyness = 100
# e_1 = results[money][2]
# e_2 = results[moneyness][3]
# # plt.yscale("log")
# for value in e_2:
#     if value == 0:
#         index = e_2.index(value)
#         index -= 4
#     else:
#         index = 50

# plt.plot(list(grid_sizes), e_2, color="tab:red", ls='--')
# plt.plot(grid_sizes, e_1, label="CN")
# plt.plot(grid_sizes[0:index], e_2[0:index], label="FTCS")
# plt.ylim((0, 0.05))
# plt.show()

fig, (ax1, ax2, ax3) = plt.subplots(
    1, 3, figsize=((12, 6)), sharex=True, sharey=True)
# 100
e_1 = results[100][2]
e_2 = results[100][3]
print("Optimal grid size for stock price 100 and CN: ",
      grid_sizes[e_1.index(min(e_1))])
print("Optimal grid size for stock price 100 and FTCS: ",
      grid_sizes[e_2.index(min(e_2))])
# plt.yscale("log")
for value in e_2:
    if value == 0:
        index = e_2.index(value)
        index -= 4
    else:
        index = 150
index = 100
# ax1.plot(list(grid_sizes)[0:index], e_2[0:index], color="tab:red", ls='--')
ax1.plot(grid_sizes, e_1, label="CN")
index = 75
ax1.plot(grid_sizes, e_2, label="FTCS")
fit = np.polyfit(grid_sizes, e_1, 2)
a = fit[0]
b = fit[1]
c = fit[2]
fit_equation = a * np.square(grid_sizes) + b * grid_sizes + c
ax1.plot(grid_sizes, fit_equation, color="tab:red")

#110

e_1 = results[110][2]
e_2 = results[110][3]
print("Optimal grid size for stock price 110 and CN: ",
      grid_sizes[e_1.index(min(e_1))])
print("Optimal grid size for stock price 110 and FTCS: ",
      grid_sizes[e_2.index(min(e_2))])
# plt.yscale("log")
for value in e_2:
    if value == 0:
        index = e_2.index(value)
        index -= 5
    else:
        index = 150
index = 100
# ax2.plot(list(grid_sizes)[0:160], e_2[0:160], color="tab:red", ls='--')
ax2.plot(grid_sizes, e_1, label="CN")
index = 75
ax2.plot(grid_sizes, e_2, label="FTCS")
fit = np.polyfit(grid_sizes, e_1, 2)
a = fit[0]
b = fit[1]
c = fit[2]
fit_equation = a * np.square(grid_sizes) + b * grid_sizes + c
ax2.plot(grid_sizes, fit_equation, color="tab:red")

#120
e_1 = results[120][2]
e_2 = results[120][3]
print("Optimal grid size for stock price 120 and CN: ",
      grid_sizes[e_1.index(min(e_1))])
print("Optimal grid size for stock price 120 and FTCS: ",
      grid_sizes[e_2.index(min(e_2))])
for value in e_2:
    if value == 0:
        index = e_2.index(value)
        index -= 4
    else:
        index = 150
index = 100
# ax3.plot(list(grid_sizes)[0:index], e_2[0:index], color="tab:red", ls='--')
ax3.plot(grid_sizes, e_1, label="CN")
index = 75
ax3.plot(grid_sizes, e_2, label="FTCS")
for ax in (ax1, ax2, ax3):
    ax.set_xlabel('Mesh size')

# plt.yscale("log")
fit = np.polyfit(grid_sizes, e_1, 2)
a = fit[0]
b = fit[1]
c = fit[2]
fit_equation = a * np.square(grid_sizes) + b * grid_sizes + c
plt.plot(grid_sizes, fit_equation, color="tab:red")
plt.plot(grid_sizes, e_1, label="CN")

ax1.title.set_text('Stock price = 100')
ax2.title.set_text('Stock price = 110')
ax3.title.set_text('Stock price = 120')
ax1.set_ylabel('Relative error')
plt.tight_layout()
# plt.savefig(f"{main_name}_convergance_multiplot_NOLOG_large_max.pdf")
plt.tight_layout()
# plt.ylim((0, 0.10))
plt.legend()
plt.show()


#--------- Plot delta as a factor of S ---
# Show the delta calculaion for different dS and different S
# Then show the true vs analytical
def disc(value):
    return np.exp(-r) * value


def delta(S, dS):
    grid_cn, value_1 = fd_cd(N, M, T, S, S_max, K, r, sigma)
    grid_cn, value_2 = fd_cd(N, M, T, S+dS, S_max, K, r, sigma)
    # return ((np.exp(value_1) - np.exp(value_2))/dS) *  np.exp(-r)
    return (abs(disc(value_1)-disc(value_2)))/dS


price = 100
delta = delta(price, 0.00001)
bs = analytical_bs(T=T, S0=price, K=K, sigma=sigma, r=r, return_delta=False)

results_delta = {}
for value in tqdm(np.linspace(100, 0.0001, 10)):
    all_errors = []
    for price in np.linspace(10, 250, 500):
        grid_cn, value_1 = fd_cd(N, M, T, price, S_max, K, r, sigma)
        grid_cn, value_2 = fd_cd(N, M, T, price+value, S_max, K, r, sigma)
        error = (abs(disc(value_1)-disc(value_2)))/value
        all_errors.append(error)
    results_delta[value] = all_errors

for dS in np.linspace(100, 0.0001, 10):
    if dS == 100 or dS == 0.0001 or dS == 44.4:
        plt.plot(np.linspace(10, 250, 500),
                 results_delta[dS], alpha=0.80, label=dS)
    else:
        plt.plot(np.linspace(10, 250, 500), results_delta[dS], alpha=0.80)
plt.xlabel("Stock price")
plt.ylabel("Delta value")
plt.legend()
plt.savefig(f"{main_name}_delta_s0_cn.pdf")


for dS in np.linspace(100, 0.0001, 10):
    if dS > 0.05:
        continue
    else:
        plt.plot(np.linspace(10, 250, 500), results_delta[dS], alpha=0.80)

plt.xlabel("Stock price")
plt.ylabel("Delta value")
plt.legend()
plt.tight_layout()
plt.savefig(f"{main_name}_delta_s0_cn_small.pdf")


# FTCS DELTA
results_delta = {}
for value in tqdm(np.linspace(100, 0.00001, 15)):
    all_errors = []
    for price in np.linspace(10, 250, 300):
        grid_cn, value_1 = ftcs_menno(N, M, T, price, S_max, K, r, sigma)
        grid_cn, value_2 = ftcs_menno(N, M, T, price+value, S_max, K, r, sigma)
        error = (abs(disc(value_1)-disc(value_2)))/value
        all_errors.append(error)
    results_delta[value] = all_errors


for dS in np.linspace(100, 0.00001, 15):
    if dS == 100 or dS == 0.00001 or dS == 44.4:
        plt.plot(np.linspace(10, 250, 300),
                 results_delta[dS], alpha=0.80, label=dS)
    else:
        plt.plot(np.linspace(10, 250, 300), results_delta[dS], alpha=0.80)
plt.xlabel("Stock price")
plt.ylabel("Delta value")
plt.legend()
plt.tight_layout()
plt.savefig(f"{main_name}_delta_s0_ftcs.pdf")


# FTCS DELTA error
results_delta = {}
for value in tqdm(np.linspace(1, 0.00001, 1)):
    all_errors = []
    for price in np.linspace(10, 250, 300):
        grid_cn, value_1 = fd_cd(N, M, T, price, S_max, K, r, sigma)
        grid_cn, value_2 = fd_cd(N, M, T, price+value, S_max, K, r, sigma)
        error = (abs(disc(value_1)-disc(value_2)))/value
        true_delta = analytical_bs(T, price, K, sigma, r, return_delta=True)
        true_error = abs(error-true_delta)
        all_errors.append(true_error)
    results_delta[value] = all_errors

for dS in np.linspace(1, 0.00001, 10):
    plt.plot(np.linspace(10, 250, 300), results_delta[dS], alpha=0.99)
plt.xlabel("Stock price")
plt.ylabel("Absolute error")
plt.legend()
plt.tight_layout()
plt.savefig(f"{main_name}_delta_absolute_error.pdf")


# FTCS DELTA
results_delta = {}
results_delta_2 = {}
for value in np.linspace(0.01, 0.01, 1):
    all_errors = []
    all_true = []
    for price in tqdm(np.linspace(10, 250, 500)):
        grid_cn, value_1 = ftcs_menno(N, 72, T, price, S_max, K, r, sigma)
        grid_cn, value_2 = ftcs_menno(N, 72, T, price+value, S_max, K, r, sigma)
        sim_delta = (abs(disc(value_1)-disc(value_2)))/value
        grid_cn, value_1 = ftcs_menno(N, 72, T, price, S_max, K, r, sigma)
        grid_cn, value_2 = ftcs_menno(N, 72, T, price-value, S_max, K, r, sigma)
        sim_delta_2 = (abs(disc(value_1)-disc(value_2)))/value
        true_delta = analytical_bs(T, price, K, sigma, r, return_delta=True)
        # print(price, value_1, value_2, true_delta)
        # true_error = sim_delta-true_delta
        # true_error = (abs(true_delta-sim_delta)/true_delta)
        all_errors.append((sim_delta+sim_delta_2)/2)
        all_true.append(true_delta)
    results_delta[value] = all_errors
    results_delta_2[value] = all_true

# for dS in np.linspace(0.000001, 0.0000001, 1):

for dS in np.linspace(0.01, 0.01, 1):
    plt.plot(np.linspace(10, 250, 500),
             results_delta_2[dS], alpha=0.99, label="True delta")
    plt.plot(np.linspace(10, 250, 500),
             results_delta[dS], alpha=0.99, label="Simulated delta")
    plt.legend()

plt.xlabel("Stock price")
plt.ylabel("Delta value")
plt.legend()
plt.savefig(f"{main_name}_true_vs_sim_delta_CN_409_final_version.pdf")
# plt.title("FTCS - M=500")
plt.tight_layout()
# plt.savefig(f"{main_name}_true_vs_sim_delta_FTCS_76_final_version.pdf")


 50%|████▉     | 497/1001 [16:56<32:36,  3.88s/it]

KeyboardInterrupt: 